In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
import time
import json
import os

# --- SETUP ---
# We will use the Gemini API for simplicity as it handles video processing easily.
# If you don_t want to paste your key every time, you can hardcode it here.
API_KEY = input("Enter your Gemini API Key: ")
genai.configure(api_key=API_KEY)

print("✅ Gemini Configured")

## 🧠 Knowledge Base
Defining the Avatars, Headlines, and Winning Structures.

In [ ]:
AVATARS = {
  "dubai_men_40": {
    "name": "DIFC Daniel",
    "pain_points": "Stress belly, low energy, time-poor, losing competitive edge against younger colleagues.",
    "desires": "Peak physical and mental performance, to look as successful as he is, to dominate in the boardroom.",
    "psych_hook": "Reclaiming physical dominance and status."
  },
  "abu_dhabi_women_50": {
    "name": "Embassy Wife Elizabeth",
    "pain_points": "Hornonal weight gain from menopause, stress eating due to social obligations, feeling invisible and that it_s _too late_ to change.",
    "desires": "To fit into her elegant gowns again, to feel confident and vibrant at diplomatic events, to restore her energy and feel feminine.",
    "psych_hook": "Conquering menopause and restoring self-worth."
  },
  "dubai_women_40": {
    "name": "Busy Mona",
    "pain_points": "Stubborn post-baby weight, no _me-time_ between school runs and family duties, feels her husband doesn_t see her the same way.",
    "desires": "To get her pre-baby body back, to feel sexy and confident, to reignite the spark in her marriage.",
    "psych_hook": "Identity reclamation and spousal attention."
  }
}

WINNING_STRUCTURES = [
    {
      "name": "The Pattern Interrupt",
      "structure": [
        { "time": "0-3s", "element": "Visual Shock / Weirdness", "purpose": "Stop the scroll immediately" },
        { "time": "3-8s", "element": "Problem Agitation", "purpose": "Call out the specific pain point" },
        { "time": "8-20s", "element": "The Mechanism", "purpose": "Show HOW the solution works" },
        { "time": "20-30s", "element": "Urgency CTA", "purpose": "Direct command to click" }
      ]
    },
    {
      "name": "The Us vs Them",
      "structure": [
        { "time": "0-5s", "element": "Split Screen Comparison", "purpose": "Visual contrast between _Old Way_ and _New Way_" },
        { "time": "5-15s", "element": "The _Aha_ Moment", "purpose": "Explain why the old way failed" },
        { "time": "15-30s", "element": "Social Proof / Results", "purpose": "Show others achieving the result" },
        { "time": "30-40s", "element": "Risk Reversal CTA", "purpose": "Guarantee or low-friction offer" }
      ]
    }
]

NEURO_HOOKS = [
    "Eye Contact: Subject looks directly at camera",
    "Movement: Action moves from Left to Right",
    "Color Contrast: High contrast text on background",
    "Facial Expression: Micro-expression of surprise"
]

## 📤 Step 1: Select Video
Make sure you have uploaded a video file to the file browser on the left.

In [ ]:
# REPLACE THIS with your actual filename
VIDEO_FILENAME = "video.mp4" 

if not os.path.exists(VIDEO_FILENAME):
    print(f"❌ Error: File _{VIDEO_FILENAME}_ not found. Please upload it to the file browser on the left.")
else:
    print(f"✅ Found file: {VIDEO_FILENAME}")
    
    print("Uploading to Gemini File API...")
    video_file = genai.upload_file(path=VIDEO_FILENAME)
    print(f"Completed upload: {video_file.uri}")

    # Wait for processing
    print("Processing video...", end="")
    while video_file.state.name == "PROCESSING":
        print(".", end="")
        time.sleep(2)
        video_file = genai.get_file(video_file.name)

    if video_file.state.name == "FAILED":
      raise ValueError(video_file.state.name)

    print("\n✅ Video is ready for analysis.")

## 🕵️ Step 2: Analyze Video Strategy
Using **Gemini 1.5 Pro** to analyze the video content.

In [ ]:
analysis_model = genai.GenerativeModel(model_name="gemini-1.5-pro")

system_prompt = f"""You are "Andromeda AI", an elite video intelligence analyst.
**MISSION:** Perform a deep forensic analysis of the video to devise a winning campaign strategy.
**KNOWLEDGE BASE:** Apply these principles: {_, _.join(NEURO_HOOKS)}.

Provide the output in strict JSON format with the following structure:
{{
  "primaryVideoFileName": "string",
  "strategyJustification": "string",
  "videoAnalyses": [
    {{
      "fileName": "string",
      "rank": number,
      "summary": "string",
      "sceneDescriptions": [ {{ "timestamp": "string", "description": "string" }} ],
      "emotionalTone": ["string"]
    }}
  ]
}}
"""

print("Analyzing video... this may take a minute...")
response = analysis_model.generate_content(
    [video_file, system_prompt],
    generation_config={"response_mime_type": "application/json"}
)

strategy = json.loads(response.text)
print(json.dumps(strategy, indent=2))

## ✍️ Step 3: Generate Ad Blueprints
Using **Gemini 1.5 Flash** to generate creative variations.

In [ ]:
# Configuration
SELECTED_AVATAR_KEY = "dubai_men_40" # Change this to _abu_dhabi_women_50_ etc.
PRODUCT_NAME = "PTD 12-Week Superhuman Program"
OFFER = "Free 30-min Consultation"

avatar = AVATARS[SELECTED_AVATAR_KEY]
generation_model = genai.GenerativeModel(model_name="gemini-1.5-flash")

creative_prompt = f"""You are an elite AI strategist.
**TARGET AVATAR:** {avatar[_name_]} (Pains: {avatar[_pain_points_]}, Desires: {avatar[_desires_]})
**PRODUCT:** {PRODUCT_NAME} | **OFFER:** {OFFER}
**STRATEGY:** {strategy[_strategyJustification_]}

Generate 5 distinct ad blueprints using these structures: {json.dumps(WINNING_STRUCTURES)}

Output JSON array:
[{{
  "variationTitle": "string",
  "headline": "string",
  "body": "string",
  "editPlan": [ {{ "timestamp": "string", "visual": "string", "overlayText": "string" }} ]
}}]
"""

print("Generating creatives...")
creatives_response = generation_model.generate_content(
    creative_prompt,
    generation_config={"response_mime_type": "application/json"}
)

creatives = json.loads(creatives_response.text)
print(f"Generated {len(creatives)} creatives.")
for i, c in enumerate(creatives):
    print(f"\n--- Creative {i+1}: {c[_variationTitle_]} ---")
    print(f"Headline: {c[_headline_]}\nBody: {c[_body_]}\nEdit Plan:")
    for scene in c[_editPlan_]:
        print(f"  [{scene[_timestamp_]}] {scene[_visual_]} (Text: {scene.get(_overlayText_, _N/A_)})")